In [1]:
#!/usr/bin/env python

This code tests the idea of random SVD. <br>
Random SVD is a way of performing fasters SVD by randomly <br>
project the data down to a lower dimension. 

**Method**<br>
Suppose we have Q matrix with orthonormal columns with the properties : <br>
$$ K \approx QQ^* K$$<br>
that is, ideally $QQ^* = I$ 

The idea of random svd, is that if we know Q, we can perform svd on <br>
the significantly smaller matrix of $Q^*K$ with the following derivation<br>
$$ K \approx Q (Q^* K)$$<br>
$$ K \approx Q (M)$$<br>
$$ K \approx Q (UΣV^T)$$

Note that $M = UΣV^T$

Notice that while $K \in \mathbb{R}^{n \times n}$ the dimension of $Q^*K$ is $\mathbb{R}^{n \times q}$ where $q$ is very small.<br>
(Note: q is normally just set to values between 10 to 20)<br>
Since $q$ is small the svd of $(Q^* K)$ is performed on a $q \times q$ matrix.<br>
This can be done very cheap. 

Once you have performed SVD on $(Q^* K) = UΣV^T$, the eigenvector the K is<br>
$$eigV(K) \approx QU$$

The key question is "How do we get Q?", this is normally done by first generating<br>
a matrix $\Omega \in \mathbb{R}^{n \times q}$ filled by normal Gaussian distribution<br>
 We project the K onto $\Omega$ <br>
$$Y = K\Omega$$

Note that $Y$ only has $q$ columns and a QR decomposition can be very cheaply conducted <br>
when there are few columns. <br>
$$ [Q,R] = qr(Y)$$

This allows us to get the Q matrix where $QQ^* = I$

In [2]:
import numpy as np
import sklearn
import sklearn.metrics
from sklearn.metrics import mean_absolute_error
from tools import *

In [3]:
X = csv_load('../dataset/wine.csv', shuffle_samples=True)
n = X.shape[0]		# number of total samples
γ = get_rbf_γ(X)	# γ used for the gaussian kerenl

In [4]:
K = sklearn.metrics.pairwise.rbf_kernel(X, gamma=γ)
Ω = np.random.randn(n,10)
Y = K.dot(Ω) #note that Y is a tall and smaller matrix compared to K, therefore QR is cheaper
print(Y.shape)

(178, 10)


In [5]:
[Q,R] = np.linalg.qr(Y)
M = Q.T.dot(K)
[Ū, Σ, Vᵀ] = np.linalg.svd(M)
U = Q.dot(Ū)		
print(Ū.shape)						#notice that Ū is very small
print(U.shape)						#notice that U is large

(10, 10)
(178, 10)


Compute the actual eigenvectors of K

In [6]:
[Λ,V] = np.linalg.eig(K)

Visually compare the two eigenvectors

In [7]:
MAE = mean_absolute_error(V[:,0:4], U[:,0:4])
print_two_matrices_side_by_side(U[30:50,0:3], V[30:50,0:3], title1='Approximate eigV', title2='Actual eigV', auto_print=True)
jupyter_print('The mean absolute error is : %.3f'% MAE)

     Approximate eigV    	Actual eigV       
[ 0.0265 -0.1271  0.105 ]	[-0.0265 -0.1272 -0.1049]
[ 0.0553 -0.1164 -0.0307]	[-0.0553 -0.1164  0.0307]
[ 0.0936  0.0288 -0.0144]	[-0.0936  0.0287  0.0145]
[ 0.0398 -0.1312  0.039 ]	[-0.0398 -0.1312 -0.039 ]
[ 0.0445 -0.1285  0.0163]	[-0.0445 -0.1285 -0.0163]
[ 0.0681 -0.0928 -0.0716]	[-0.0681 -0.0927  0.0715]
[ 0.0915 -0.0098 -0.0668]	[-0.0915 -0.0099  0.0668]
[ 0.0892  0.0525  0.0371]	[-0.0892  0.0524 -0.0371]
[ 0.0697  0.0677  0.1033]	[-0.0697  0.0678 -0.1034]
[ 0.0857 -0.0397 -0.0864]	[-0.0857 -0.0397  0.0863]
[ 0.0739 -0.0784 -0.0831]	[-0.0739 -0.0784  0.083 ]
[ 0.0185 -0.1133  0.137 ]	[-0.0186 -0.1134 -0.1369]
[ 0.0602  0.0657  0.1132]	[-0.0602  0.0656 -0.1131]
[ 0.0933  0.0346 -0.0038]	[-0.0933  0.0345  0.0038]
[ 0.0828  0.0628  0.07  ]	[-0.0828  0.0628 -0.07  ]
[ 0.0737 -0.0785 -0.0832]	[-0.0737 -0.0785  0.0832]
[ 0.0906 -0.016  -0.0721]	[-0.0906 -0.016   0.0721]
[ 0.068  -0.093  -0.0712]	[-0.0679 -0.093   0.0711]
[ 0.0724 -0.0822 -0